# 시장별 우세 품목 확인 (13_20년도_5대시장_거래량_통합)

### import

In [ ]:
import pandas as pd
import numpy as np
import os

import datetime
from datetime import date, timedelta

import warnings
warnings.filterwarnings(action='ignore')

### 도매 가격 종합 데이터셋 input

In [ ]:
path = '../data/거래량/원본/'
file_list = os.listdir (path)
file_list_py = [file for file in file_list if '.xlsx' in file]
file_list_py

### 데이터셋 concat.

In [ ]:
반입물량 = pd.DataFrame()

for files in file_list_py :
    df = pd.read_excel(path + files)
    
    df.drop([0, 1], inplace = True)
    df.drop(df[df['거래년월일'] == '소계'].index, inplace = True)
    
    df.reset_index(drop = True, inplace = True)
    df.rename(columns = {'거래년월일':'날짜'}, inplace = True)
    
    df['날짜'] = pd.to_datetime(df['날짜'])
    df['거래량'] = df['거래량'].str.replace(',', '')
    df.drop(['시장코드', '출하지코드', '품목코드'], axis = 1, inplace = True)    
    
    mapDict = {'봄배추' : '봄',
               '고냉지배추' : '여름', 
               '여름배추' : '여름', 
               '김장(가을)배추' : '가을', 
               '월동배추' : '겨울'}
    df['품목명'] = df['품목명'].apply(lambda x : mapDict[x])
    
    반입물량 = pd.concat([반입물량, df])
    
반입물량

### 시장별 각 품목별 총 거래량 합산

In [ ]:
반입물량['거래량'] = 반입물량['거래량'].apply(lambda x : int(x))
반입물량_시장별 = pd.DataFrame(반입물량.groupby(['날짜', '품목명', '시장명'])['거래량'].sum()).reset_index()
반입물량_시장별

In [ ]:
반입물량_시장별['시장명'].unique()

In [ ]:
반입물량_시장별['시장명'] = 반입물량_시장별['시장명'].apply(lambda x : x[:2])
반입물량_시장별.columns = ['날짜', '품명', '시장', '거래량']
반입물량_시장별 = 반입물량_시장별[['날짜', '시장', '품명', '거래량']]
반입물량_시장별

### 데이터셋 파일 저장

In [ ]:
반입물량_시장별.to_csv('../data/거래량/13_21년도_5대시장_거래량_종합.csv', encoding = 'utf-8-sig')